In [2]:
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 500)

In [3]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)

In [4]:
from utils import UtilsKy
from analyzer import HelperAnalyzer, AnalyzerPrediction

In [5]:
# for autoreload modules
%load_ext autoreload
%autoreload 2

In [6]:
# pytorch mlp for binary classification
from numpy import vstack
from pandas import read_csv
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch import Tensor
from torch.nn import Linear
from torch.nn import ReLU
from torch.nn import Sigmoid
from torch.nn import Module
from torch.optim import SGD
from torch.nn import BCELoss
from torch.nn.init import kaiming_uniform_
from torch.nn.init import xavier_uniform_

In [47]:
def minMaxScaler_own(teach, test):
    teach = teach.copy()
    test = test.copy()
    col_names = list(teach)
    for col in col_names:
        x_min = teach[col].min(axis=0)
        x_max = teach[col].max(axis=0)
        teach[col] = (teach[col] - x_min) / (x_max - x_min)
        test[col] = (test[col] - x_min) / (x_max - x_min)
        test[col] = np.where(test[col] > 1,  1, test[col])
        test[col] = np.where(test[col] < 0 , 0, test[col])        
    return teach, test

In [48]:
db_teach_orig = pd.read_csv( UtilsKy.DB_TEACH_KYW3, dtype=str, encoding='cp1251')

In [49]:
db_test_orig = pd.read_csv(UtilsKy.DB_TEST_KYW3, dtype=str, encoding='cp1251')

In [50]:
white = pd.read_csv(UtilsKy.WHITE_KYW3 , dtype=str)

In [51]:
df = db_teach_orig.copy()
db_test = db_test_orig.copy()

In [52]:
str(list(df))

"['amount', 'amount_deviation', 'bank_currency', 'bin', 'city', 'count_months_to_end_card', 'day_of_week', 'gender2', 'hour', 'id', 'is_city_resolved', 'is_gender_undefined', 'latitude', 'longitude', 'order_id', 'phone_2_norm', 'status']"

In [53]:
df.head()

,amount,amount_deviation,bank_currency,bin,city,count_months_to_end_card,day_of_week,gender2,hour,id,is_city_resolved,is_gender_undefined,latitude,longitude,order_id,phone_2_norm,status
0,1.00,-3,112,547087,Stuttgart,29,3,undefined,09,547087******1680_2022-04,1,1,48.7667,9.1833,49219577955111383,55,0
1,25.00,-2,840,517148,Kapaa,41,3,f,12,517148******7320_2023-04,1,0,22.0837,-159.3553,49221760925063337,08,0
2,21.18,-2,840,546540,undefined,44,3,m,15,546540******9404_2023-07,0,0,38.0000,-97.0000,49224688788195598,40,0
3,200.00,2,840,510805,Oakland,40,3,m,15,510805******1186_2023-03,1,0,37.8116,-122.2420,49224696921328680,10,0
4,2.38,-3,76,544731,Rio De Janeiro,80,3,undefined,15,544731******7316_2026-07,1,1,-22.9000,-43.2333,49224839756465248,19,0


In [54]:
numeric_cols = ['amount', 'amount_deviation', 'bank_currency', 'bin', 'count_months_to_end_card', 'day_of_week', 'hour', 'is_city_resolved', 
                'is_gender_undefined', 'latitude', 'longitude', 'phone_2_norm']

In [55]:
df.status = df.status.astype('float32')

In [56]:
df.latitude = df.latitude.apply(lambda x: 0 if x == 'undefined' else x)
df.longitude = df.longitude.apply(lambda x: 0 if x == 'undefined' else x)

In [57]:
df[numeric_cols] = df[numeric_cols].astype('float32').copy()

In [58]:
db_test.status = df.status.astype('float32')
db_test.latitude = db_test.latitude.apply(lambda x: 0 if x == 'undefined' else float(x))
db_test.longitude = db_test.longitude.apply(lambda x: 0 if x == 'undefined' else float(x))
db_test[numeric_cols] = db_test[numeric_cols].astype('float32').copy()

In [59]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [60]:
scaler = 'own'
if scaler != 'own':
    df[numeric_cols] = scaler.fit_transform(df[numeric_cols])
    db_test[numeric_cols] = scaler.fit_transform(db_test[numeric_cols])

In [62]:
if scaler =='own' :
    df[numeric_cols], db_test[numeric_cols] = minMaxScaler_own(df[numeric_cols], db_test[numeric_cols])

In [67]:
col_names = numeric_cols + ['status']
df = df[col_names]

In [68]:
# dataset definition
class CSVDataset(Dataset):
    # load the dataset
    def __init__(self, df): # path
        # store the inputs and outputs
        self.X = df.values[:, :-1]
        self.y = df.values[:, -1]
        # ensure input data is floats
        self.X = self.X.astype('float32')
        # label encode target and ensure the values are floats
        self.y = LabelEncoder().fit_transform(self.y)
        self.y = self.y.astype('float32')
        self.y = self.y.reshape((len(self.y), 1))
 
    # number of rows in the dataset
    def __len__(self):
        return len(self.X)
 
    # get a row at an index
    def __getitem__(self, idx):
        return [self.X[idx], self.y[idx]]
 
    # get indexes for train and test rows
    def get_splits(self, n_test=0.05):
        # determine sizes
        test_size = round(n_test * len(self.X))
        train_size = len(self.X) - test_size
        # calculate the split
        return random_split(self, [train_size, test_size])

In [69]:
# model definition
class MLP(Module):
    # define model elements
    def __init__(self, n_inputs):
        super(MLP, self).__init__()
        # input to first hidden layer
        self.hidden1 = Linear(n_inputs, 10)
        kaiming_uniform_(self.hidden1.weight, nonlinearity='relu')
        self.act1 = ReLU()
        # second hidden layer
        self.hidden2 = Linear(10, 6)
        kaiming_uniform_(self.hidden2.weight, nonlinearity='relu')
        self.act2 = ReLU()
        # third hidden layer and output
        self.hidden3 = Linear(6, 1)
        xavier_uniform_(self.hidden3.weight)
        self.act3 = Sigmoid()
 
    # forward propagate input
    def forward(self, X):
        # input to first hidden layer
        X = self.hidden1(X)
        X = self.act1(X)
         # second hidden layer
        X = self.hidden2(X)
        X = self.act2(X)
        # third hidden layer and output
        X = self.hidden3(X)
        X = self.act3(X)
        return X

In [70]:
# prepare the dataset
def prepare_data(df):
    # load the dataset
    dataset = CSVDataset(df)
    # calculate split
    train, test = dataset.get_splits()
    # prepare data loaders
    train_dl = DataLoader(train, batch_size=32, shuffle=True)
    test_dl = DataLoader(test, batch_size=1024, shuffle=False)
    return train_dl, test_dl

In [71]:
# train the model
def train_model(train_dl, model, n_epoch=100):
    # define the optimization
    criterion = BCELoss()
    optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9)
    # enumerate epochs
    for epoch in range(n_epoch):
        # enumerate mini batches
        for i, (inputs, targets) in enumerate(train_dl):
            # clear the gradients
            optimizer.zero_grad()
            # compute the model output
            yhat = model(inputs)
            # calculate loss
            loss = criterion(yhat, targets)
            # credit assignment
            loss.backward()
            # update model weights
            optimizer.step()


In [72]:
# evaluate the model
def evaluate_model(test_dl, model):
    predictions, actuals = list(), list()
    for i, (inputs, targets) in enumerate(test_dl):
        # evaluate the model on the test set
        yhat = model(inputs)
        # retrieve numpy array
        yhat = yhat.detach().numpy()
        actual = targets.numpy()
        actual = actual.reshape((len(actual), 1))
        # round to class values
        yhat = yhat.round()
        # store
        predictions.append(yhat)
        actuals.append(actual)
    predictions, actuals = vstack(predictions), vstack(actuals)
    # calculate accuracy
    acc = accuracy_score(actuals, predictions)
    return acc

In [73]:
#  make a class prediction for one row of data
def predict(row, model):
    # convert row to data
    row = Tensor([row])
    # make prediction
    yhat = model(row)
    # retrieve numpy array
    yhat = yhat.detach().numpy()
    return yhat

In [74]:
def get_db_test_prediction(db_test, model,numeric_cols):
    predictions = list()
    test = db_test[numeric_cols].values
    for row in test:
        predictions.append(predict(row, model)[0][0])
    return predictions    
        

In [75]:
train_dl, test_dl = prepare_data(df)
print(len(train_dl.dataset), len(test_dl.dataset))

411754 21671


In [76]:
# define the network
model = MLP(12)

In [77]:
# evaluate the model
acc = evaluate_model(test_dl, model)
print('Accuracy: %.3f' % acc)

Accuracy: 0.965


In [78]:
analyzer_prediction =  AnalyzerPrediction(db_teach_orig, db_test_orig, white)

In [79]:
result_df_amount = None

In [80]:
for n_epoch in (3, 4, 5, 6, 10):
    # train the model
    train_model(train_dl, model, n_epoch)
    
    test_probability = get_db_test_prediction(db_test, model,numeric_cols)
    db_test_orig["probability"] = test_probability
    
    description = '10-6-1 ' + 'amount ' + 'n_ep={}'.format(n_epoch)          
    result_df_amount = analyzer_prediction.get_table_prediction(description=description, result_df=result_df_amount, metric="amount")
    print( 'n_epoch={}'. format(n_epoch ))

n_epoch=3
n_epoch=4
n_epoch=5
n_epoch=6
n_epoch=10


In [81]:
n = result_df_amount.shape[0]
sub_rows = list(range(n))[::2]
stat_best = result_df_amount.copy().iloc[sub_rows,:]

col_names = [col for col in stat_best.columns if col.startswith('p_') ] 
stat_best.loc[:, col_names] = stat_best.loc[:, col_names].astype(float)
stat_best = stat_best.sort_values(by="rating", ascending=False)

In [82]:
stat_best.iloc[:,:15]

,description,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_10,p_20,rating,n_white_list,n_test_in_wl,n_test_bad_in_wl,amount_test_in_wl
4,10-6-1 amount n_ep=5,3.68,9.48,11.71,15.95,19.71,22.67,24.83,29.63,53.35,143.51,1019125,22992,34,1640236.53
2,10-6-1 amount n_ep=4,5.13,9.50,14.49,14.83,17.45,18.64,20.22,26.08,51.29,138.77,1019125,22992,34,1640236.53
8,10-6-1 amount n_ep=10,4.49,8.10,10.08,14.25,16.01,18.90,21.39,30.13,54.29,137.35,1019125,22992,34,1640236.53
6,10-6-1 amount n_ep=6,2.89,6.43,10.61,12.39,16.68,20.81,24.20,29.58,54.93,133.51,1019125,22992,34,1640236.53
0,10-6-1 amount n_ep=3,2.03,4.65,7.15,9.67,12.18,13.61,17.47,29.57,51.54,116.79,1019125,22992,34,1640236.53
